In [1]:
# Lê uma imagem multiband e normaliza com o StandardScaler

In [12]:
import os
os.chdir('/mnt/mint_files/TCC/main_code/')

In [13]:
import numpy as np
from tensorflow import keras
from glob import glob
import rasterio as rio
from pyrsgis import raster
import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from PIL import Image as im

In [14]:
recreioFile = 'images/lagoa_freitas/2015/lagoa_freitas_v1_multiband_6.TIF'
# recreioFile = 'images/lagoa_freitas/land_5/2011/lagoa_freitas_v1_multiband_6.tif'
recreioDs, recreioFeatures = raster.read(recreioFile, bands='all')
print("Recreio multispectral image shape: ", recreioFeatures.shape)

Recreio multispectral image shape:  (6, 360, 500)


In [15]:
x = np.moveaxis(recreioFeatures, 0, -1)
x.reshape(-1, recreioFeatures.shape[0]).shape, recreioFeatures.shape[1] * recreioFeatures.shape[2]
X_data = x.reshape(-1, recreioFeatures.shape[0])
X_data.shape

(180000, 6)

In [16]:
print(np.min(X_data))
print(np.max(X_data))

5110
23510


In [17]:
# Descomente abaixa qual a normalização que será utilizada

# normalização fórmula
#scaler = MinMaxScaler().fit(X_data)
#X_scaled = scaler.fit_transform(X_data)
#X_scaled.shape

# normalização direta
X_scaled = X_data / 65535.0

In [18]:
X_scaled = X_scaled.reshape((X_scaled.shape[0], 1, X_scaled.shape[1]))
print(X_scaled.shape)

(180000, 1, 6)


In [19]:
valor_cortes = [0.005,0.01, 0.05, 0.10, 0.20, 0.30, 0.40, 0.50, 0.60, 0.70, 0.80, 0.90, 0.95, 0.99, 0.999, 0.9995, 0.9999]
# valor_cortes = [0.995, 0.999, 0.9995, 0.9999]
# total_cortes = 5
arr_ep = ['2', '5', '10', '20', '25', '100']
# arr_ep = ['2']
# arr_ep = ['100']

# land8
featuresRecreio_many = [
    ["234", np.copy(X_scaled[:,:,[1,2,3]])],
    ["345", np.copy(X_scaled[:,:,[2,3,4]])],
    ["256", np.copy(X_scaled[:,:,[1,4,5]])],
    #["134", np.copy(featuresRecreio[:,:,[0,2,3]])],
    ["allb", np.copy(X_scaled)],
]

# featuresRecreio_many = [["allb", featuresRecreio]]
total_features = 4
# 
for ep in arr_ep:
    for i in range(total_features):
        print(f'modelos/my_model_{featuresRecreio_many[i][0]}_{ep}.h5')
        modelo = keras.models.load_model(f'modelos/my_model_{featuresRecreio_many[i][0]}_{ep}.h5')
        predicao = modelo.predict(featuresRecreio_many[i][1])
        visualiza = predicao[:,1]
        for valCorte in valor_cortes:
            resultPredict = (visualiza>valCorte).astype(int)
            arrayOfnumeros = np.array(resultPredict)
            # (360, 500) - lagoa v1
            # (200, 150) - lagoa v2
            # (300, 650) - barra
            arrayOfnumeros = np.reshape(arrayOfnumeros, (360, 500))
            data = im.fromarray((arrayOfnumeros*255).astype(np.uint8))
            data.save(f'images/output_4_land8/{ep}_epochs/b_{featuresRecreio_many[i][0]}_{str(valCorte).replace(".", "_")}.png')

modelos/my_model_234_2.h5
modelos/my_model_345_2.h5
modelos/my_model_256_2.h5
modelos/my_model_allb_2.h5
modelos/my_model_234_5.h5
modelos/my_model_345_5.h5
modelos/my_model_256_5.h5
modelos/my_model_allb_5.h5
modelos/my_model_234_10.h5
modelos/my_model_345_10.h5
modelos/my_model_256_10.h5
modelos/my_model_allb_10.h5
modelos/my_model_234_20.h5
modelos/my_model_345_20.h5
modelos/my_model_256_20.h5
modelos/my_model_allb_20.h5
modelos/my_model_234_25.h5
modelos/my_model_345_25.h5
modelos/my_model_256_25.h5
modelos/my_model_allb_25.h5
modelos/my_model_234_100.h5
modelos/my_model_345_100.h5
modelos/my_model_256_100.h5
modelos/my_model_allb_100.h5
